<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-2---Elements-of-Matrix-Theory" data-toc-modified-id="Chapter-2---Elements-of-Matrix-Theory-1">Chapter 2 - Elements of Matrix Theory</a></span><ul class="toc-item"><li><span><a href="#2.1.2-The-Jordan-Normal-Form" data-toc-modified-id="2.1.2-The-Jordan-Normal-Form-1.1">2.1.2 The Jordan Normal Form</a></span><ul class="toc-item"><li><span><a href="#Example-2.5-Revisiting-the-wireless-sensor-network-example" data-toc-modified-id="Example-2.5-Revisiting-the-wireless-sensor-network-example-1.1.1">Example 2.5 Revisiting the wireless sensor network example</a></span></li><li><span><a href="#NumPy/-SciPy-approach" data-toc-modified-id="NumPy/-SciPy-approach-1.1.2">NumPy/ SciPy approach</a></span></li></ul></li></ul></li></ul></div>

In [1]:
%matplotlib widget

# Import packages
import scipy.linalg as spla
import numpy as np
import matplotlib.pyplot as plt


# For interactive graphs
import ipywidgets as widgets

# Settings
custom_figsize = (6, 4)  # Might need to change this value to fit the figures to your screen

In [14]:
def matprint(mat, fmt="g"):
    """
    Own defined function to beautiful print matrices in the output.
    """
    # Handling 1d-arrays
    if np.ndim(mat) == 1:
        mat = mat.reshape(mat.shape[0], 1)
    # Handling column spaces
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]

    for x in mat:
        for i, y in enumerate(x):
            if i == 0:
                print(("|  {:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
            else:                    
                print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("|")
        

def plot_spectrum(M):
    """
    Scatter plot of eigs in complex plane overalyed over unit circle
    """
    eigvals = np.linalg.eig(M)[0]
    reals = [i.real for i in eigvals]
    imags = [i.imag for i in eigvals]
    
    fig, ax = plt.subplots(figsize=(4,4))
    plt.scatter(reals, imags)
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    
    circle=patches.Circle((0,0),radius=1,alpha=.1)
    ax.add_patch(circle)
    return eigvals, fig, ax

**TODO:**

Chapter 2:
- randomly generate a matrix, compute Jordan form, plot spectrum
- randomly generate a matrix and plot the Gersgorin disks
- compute powers of a primitive row-stochastic matrix and show it converges to rank 1
- illustrate dynamics of algorithms 2.17 and 2.18

# Chapter 2 - Elements of Matrix Theory
These Jupyter Notebook scripts contain some examples, visualization and supplements accompanying the book "Lectures on Network Systems" by Francesco Bullo http://motion.me.ucsb.edu/book-lns/. These scripts are published with the MIT license. **Make sure to run the first cell above to import all necessary packages and functions and adapt settings in case.** After that, you can jump to each example and follow the description there to execute cell by cell (Tip: Use the shortcut Shift+Enter to execute each cell). Most of the functions are kept in separate files to keep this script neat.

## 2.1.2 The Jordan Normal Form
### Example 2.5 Revisiting the wireless sensor network example
The following cells are showing the computation of the Jordan Normal Form $J$, the invertible transformation matrix $T$ and some of its dependencies. 

In [3]:
# Defining the A matrix again
A = np.array([[1/2, 1/2, 0., 0.],
              [1/4, 1/4, 1/4, 1/4],
              [0., 1/3, 1/3, 1/3],
              [0., 1/3, 1/3, 1/3]
])

There is the possibility to calculate the Jordan Normal Form directly with the package SymPy https://simpy.readthedocs.io/en/latest/. However, we are determining the Jordan Normal Form via determining the generalized eigenvectors (read more for literature recommendations about generalized eigenvectors in the book) with the SciPy package first to discuss some possibilities. From the documentation of scipy.linalg.eig: *'Solve an ordinary or generalized eigenvalue problem of a square matrix.'*

### NumPy/ SciPy approach

In [7]:
# Right eigenvectors
lambdas, eigv = spla.eig(A)

# Left eigenvectors
lambdas2, eigw = spla.eig(A.T)

Due to numerical instabilities, the zero values are not reflected and it can be seen, how the expected eigenvalue of 1 is not precise. The zeros can be fixed with:

In [8]:
def correct_close_to_zero(M, tol=1e-12):
    M.real[abs(M.real) < tol] = 0.0
    if M.imag.any():
        M.imag[abs(M.imag) < tol] = 0.0
    return M

eigv_cor = correct_close_to_zero(eigv)
eigw_cor = correct_close_to_zero(eigw)
lambdas_cor = correct_close_to_zero(lambdas)
lambdas2_cor = correct_close_to_zero(lambdas2)

print("Right eigenvectors:")
matprint(eigv_cor)
print("\n")
print("Left eigenvectors:")
matprint(eigw_cor)
print("\n")
print("Eigenvalues (right):")
matprint(lambdas_cor)
print("\n")
print("Eigenvalues (left) for matching later:")
matprint(lambdas2_cor)

Right eigenvectors:
|  -0.5  0.855025   0.555542          0  |
|  -0.5  0.110013  -0.719612          0  |
|  -0.5  -0.35835   0.294561  -0.707107  |
|  -0.5  -0.35835   0.294561   0.707107  |


Left eigenvectors:
|  0.324443  -0.733894  -0.333766          0  |
|  0.648886  -0.188856   0.864677          0  |
|  0.486664   0.461375  -0.265456  -0.707107  |
|  0.486664   0.461375  -0.265456   0.707107  |


Eigenvalues (right):
|          1+0j  |
|   0.564333+0j  |
|  -0.147667+0j  |
|          0+0j  |


Eigenvalues (left) for matching later:
|          1+0j  |
|   0.564333+0j  |
|  -0.147667+0j  |
|          0+0j  |


There are two options now for $T^{-1}$: Taking the inverse of the right eigenvectors (which contains again numerical instabilities) or building it from the left eigenvectors, what would include some sorting to match the eigenvalue order from the  right eigenvector (often it is the case, that they are already aligned since calling scipy.linalg.eig twice on a matrix with the same eigenvalues).

In [9]:
T = eigv_cor.copy()
# Sorting if necessary, remember to use transpose, since in T^-1 the rows represent the left eigenvectors.
Tinv = eigw_cor.T.copy()

Now we can simply compute J, when compared, is fairly close to the solution in the book, however, due to numerical intabilities not precise. Further on, the order of the eigenvalues might be different than the on from the book.

In [13]:
J = correct_close_to_zero(Tinv@A@T)
print("Jordan Normal Form via SciPy/Numpy:")
matprint(J)

Jordan Normal Form via SciPy/Numpy:
|  -0.973329         0         0  0  |
|          0  -0.55245         0  0  |
|          0         0  0.142357  0  |
|          0         0         0  0  |
